In [56]:
#!pip install requests2
#!pip install beautifulsoup4
#!pip install selenium

In [57]:
#import time
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
from bs4 import BeautifulSoup
import json
import urllib.parse

In [115]:
url = 'https://newsapi.org/v2/everything?'
api_key = '126614c036da4f5ba425e28b9b04b12b'
busca = 'bolsonaro'

In [128]:
parametros = {
    'q' : busca,
    'pageSize' : 6,
    'apiKey' : api_key,
    'language' : 'pt'
}

In [131]:
#chamando a news api
resp = requests.get(url, params = parametros)
print(resp)
resp_json = resp.json()

#resp_json

<Response [200]>


In [132]:
#pegando a parte da resposta que interessa
print(resp_json.keys())
resp_articles = resp_json['articles']
len(resp_articles)

dict_keys(['status', 'totalResults', 'articles'])


6

In [133]:
#transformando os artigos de json pra df
data = json.loads(json.dumps(resp_articles))
dfArtigos = json_normalize(data)
type(dfArtigos)
dfArtigos

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Robert Mackey,Tratamento experimental para covid-19 elogiado...,Trump e Bolsonaro apostam na hidroxicloroquina...,https://theintercept.com/2020/04/08/tratamento...,https://theintercept.com/wp-uploads/sites/1/20...,2020-04-08T18:56:57Z,Um tratamento experimental\r\n para o covid-19...,None,Theintercept.com
1,Fernanda Cimini,Coronavírus: os 15 dias de brigas políticas qu...,Brasil pode seguir o mesmo caminho com Bolsona...,https://theintercept.com/2020/03/24/coronaviru...,https://theintercept.com/wp-uploads/sites/1/20...,2020-03-24T21:14:56Z,A primeira pergunta\r\n que surge ao olharmos ...,None,Theintercept.com
2,João Filho,Troca de Mandetta por Teich mostra que Bolsona...,Todas suas ações durante a pandemia do coronav...,https://theintercept.com/2020/04/19/coronaviru...,https://theintercept.com/wp-uploads/sites/1/20...,2020-04-19T04:03:01Z,Foto: Evaristo Sa/AFP via Getty Images\r\nNa ú...,None,Theintercept.com
3,Leandro Demori,Todos viveremos a batalha de Milão,Bolsonaro jogou suas cartas na mesa. Agora tod...,https://theintercept.com/2020/03/28/coronaviru...,https://theintercept.com/wp-uploads/sites/1/20...,2020-03-28T17:06:03Z,Este texto foi publicado originalmente na news...,None,Theintercept.com
4,João Filho,Coronavírus: mentiras fabricadas pelo ‘gabinet...,Como não tem compromisso com a realidade dos f...,https://theintercept.com/2020/04/12/gabinete-o...,https://theintercept.com/wp-uploads/sites/1/20...,2020-04-12T04:03:08Z,Foto: Carolina Antunes/PR\r\nBolsonaro segue c...,None,Theintercept.com
5,Bruno Sousa,"Coronavírus: ‘na zona norte do Rio, o discurso...","Aparentemente, o presidente conseguiu o que qu...",https://theintercept.com/2020/03/26/coronaviru...,https://theintercept.com/wp-uploads/sites/1/20...,2020-03-26T21:37:05Z,"Vista da favela do Complexo do Chapadão, zona ...",None,Theintercept.com


In [134]:
dfConteudo = pd.DataFrame(columns=['nr_noticia', 'texto', 'tamanho', 'origem'])
contador = 0

for url in dfArtigos['url']:
    print(url)
    article = requests.get(url)
    article_content = article.content
    soup_article = BeautifulSoup(article_content, 'html5lib')
    body = soup_article.find_all('body')
    p = body[0].find_all('p')
    for y in p:
        text = y.get_text().strip()
        if(len(text) > 0):
            dfConteudo = dfConteudo.append({"nr_noticia":contador, "texto":text, "tamanho":len(text), "origem":url}, ignore_index=True)
    
    contador = contador + 1

print('concluido')

https://theintercept.com/2020/04/08/tratamento-hidroxicloroquina-covid-19-coronavirus-trump-bolsonaro/
https://theintercept.com/2020/03/24/coronavirus-poltica-italia/
https://theintercept.com/2020/04/19/coronavirus-mandetta-teich/
https://theintercept.com/2020/03/28/coronavirus-batalha-milao-bolsonaro/
https://theintercept.com/2020/04/12/gabinete-odio-coronavirus-bolsonaro/
https://theintercept.com/2020/03/26/coronavirus-bolsonaro-zona-norte-rio-de-janeiro/
concluido


In [109]:
noticia = dfConteudo[dfConteudo['nr_noticia']==0]
print(noticia)
noticiaFiltrado = noticia[noticia['tamanho'] > 50]

textoNoticia = ""
for nf in noticiaFiltrado['texto']:
    textoNoticia = textoNoticia + "\n" + nf

    ##############LEMBRAR DE PEGAR TITULOS TAMBEM
print('\nqtd caracteres na noticia: ' + str(len(textoNoticia)))
print(textoNoticia)

   nr_noticia                                              texto tamanho  \
0           0       The Intercept is a First Look Media Company.      44   
1           0  Um tratamento experimental para o covid-19 def...     438   
2           0  De acordo com o autor principal do artigo, o D...     584   
3           0  O presidente rejeitou repetidas vezes as adver...     542   
4           0  "What do you have to lose? Take it. I really t...     199   
5           0  “O que vocês tem a perder? Tomem. Eu realmente...     174   
6           0  Quando o presidente mencionou que pessoas com ...     202   
7           0  Segundo Stecker, o perigo aumenta pelo fato de...     566   
8           0  Na conferência de sábado, Trump também sugeriu...     404   
9           0  Watch Trump move seamlessly from saying "there...     283   
10          0  Assista a Trump passar sem rodeios de dizer qu...     302   
11          0  “Pessoas com lúpus, incluindo aquelas fazendo ...     696   
12          